In [1]:
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset

from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T

from mincut_pool_net import Net

In [2]:
max_nodes = 111
torch.manual_seed(12315)

dataset = TUDataset(root='./tmp', name='NCI1', transform=T.ToDense(max_nodes))
dataset = dataset.shuffle()
dataset_length = len(dataset)

# split dataset into 3 parts

In [3]:
D_target = dataset[:int(0.4 * dataset_length)]
D_aux = dataset[int(0.4 * dataset_length): int(0.7 * dataset_length)]
G_target = dataset[int(0.7 * dataset_length):]

D_target_train = D_target[:int(0.6 * len(D_target))]
D_target_test = D_target[int(0.6 * len(D_target)):]

In [4]:
class TrainSet(Dataset):
    def __init__(self, data):
        self.data = data

    def __getitem__(self, item):
        return self.data[item].x, self.data[item].adj, self.data[item].y

    def __len__(self):
        return len(self.data)

In [5]:
train_dataset = TrainSet(D_target_train)
train_loader = DataLoader(train_dataset, batch_size=200, shuffle=True)

test_dataset = TrainSet(D_target_test)
test_loader = DataLoader(test_dataset)

In [6]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(dataset.num_features, dataset.num_classes).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5, weight_decay=1e-4)

In [7]:
def train():
    model.train()
    loss_all = 0

    for data in train_loader:
        x = data[0].to(device)
        adj = data[1].to(device)
        y = data[2].to(device)

        optimizer.zero_grad()
        out, mc_loss, o_loss, _ = model(x, adj)
        loss = F.nll_loss(out, y.view(-1)) + mc_loss + o_loss
        loss.backward()
        loss_all += y.size(0) * float(loss)
        optimizer.step()

    return loss_all / len(train_dataset)

In [8]:
@torch.no_grad()
def test(loader):
    model.eval()
    correct = 0
    loss_all = 0

    for data in loader:
        x = data[0].to(device)
        adj = data[1].to(device)
        y = data[2].to(device)
        pred, mc_loss, o_loss, _ = model(x, adj)
        loss = F.nll_loss(pred, y.view(-1)) + mc_loss + o_loss
        loss_all += y.size(0) * float(loss)
        correct += int(pred.max(dim=1)[1].eq(y.view(-1)).sum())

    return loss_all / len(loader.dataset), correct / len(loader.dataset)

In [9]:
train_loss = test_loss = float('inf')
best_train_accu = best_test_accu = 0

for epoch in range(1, 500):
    train_loss = train()
    _, train_accu = test(train_loader)
    test_loss, test_accu = test(test_loader)
    if train_accu > best_train_accu:
        best_train_accu = train_accu
    if test_accu > best_test_accu:
        best_test_accu = test_accu
    print(
        f'Epoch: {epoch:04d}, Train Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Train Accu: {train_accu:.4f}, Test Accu: {test_accu:.4f}, Best Train Accu: {best_train_accu:.4f}, Best Test Accu: {best_test_accu:.4f}')

Epoch: 0001, Train Loss: 1.2840, Test Loss: 1.2812, Best Train Accu: 0.6004, Best Test Accu: 0.5897
Epoch: 0002, Train Loss: 1.2794, Test Loss: 1.2800, Best Train Accu: 0.6004, Best Test Accu: 0.5897
Epoch: 0003, Train Loss: 1.2769, Test Loss: 1.2781, Best Train Accu: 0.6004, Best Test Accu: 0.5897
Epoch: 0004, Train Loss: 1.2746, Test Loss: 1.2759, Best Train Accu: 0.6004, Best Test Accu: 0.5897
Epoch: 0005, Train Loss: 1.2719, Test Loss: 1.2726, Best Train Accu: 0.6004, Best Test Accu: 0.5897
Epoch: 0006, Train Loss: 1.2695, Test Loss: 1.2689, Best Train Accu: 0.6197, Best Test Accu: 0.6049
Epoch: 0007, Train Loss: 1.2668, Test Loss: 1.2660, Best Train Accu: 0.6197, Best Test Accu: 0.6109
Epoch: 0008, Train Loss: 1.2644, Test Loss: 1.2637, Best Train Accu: 0.6197, Best Test Accu: 0.6109
Epoch: 0009, Train Loss: 1.2620, Test Loss: 1.2612, Best Train Accu: 0.6217, Best Test Accu: 0.6155
Epoch: 0010, Train Loss: 1.2595, Test Loss: 1.2583, Best Train Accu: 0.6217, Best Test Accu: 0.6155


KeyboardInterrupt: 

In [ ]:
data_save_path = "NCI_model_mincut_pool.pt"
torch.save(model, data_save_path)